<a href="https://colab.research.google.com/github/enoosoft/ai_fsec/blob/master/colab/Day_3_01_Movie_Review_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

머신 러닝에서 텍스트 분류를 연습하기 위해 자주 사용하는 데이터가 있습니다. 바로 영화 사이트 IMDB의 리뷰 데이터입니다. 이 데이터는 리뷰에 대한 텍스트와 해당 리뷰가 긍정인 경우 1을 부정인 경우 0으로 표시한 레이블로 구성된 데이터입니다.

스탠포드 대학교에서 2011년에 낸 논문에서 이 데이터를 소개하였으며, 당시 이 데이터를 훈련 데이터와 테스트 데이터를 50:50대 비율로 분할하여 88.89%의 정확도를 얻었다고 소개하고 있습니다.

논문 링크 : http://ai.stanford.edu/~amaas/papers/wvSent_acl2011.pdf

파이토치에서는 해당 IMDB 영화 리뷰 데이터를 바로 다운로드 할 수 있도록 지원하고 있습니다. 파이토치로부터 해당 데이터를 다운로드 받아 감성 분류를 수행하는 모델을 만들어보겠습니다.

# 1. 셋팅 하기

우선 필요한 도구들을 임포트합니다.

In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext import data, datasets
import random

랜덤 시드를 고정시켜줍니다.

In [ ]:
SEED = 5
random.seed(SEED)
torch.manual_seed(SEED)

하이퍼파라미터들을 변수로 정의합니다.

In [ ]:
# 하이퍼파라미터
BATCH_SIZE = 64
lr = 0.001
EPOCHS = 10

현 환경에서 GPU를 사용 가능하면 GPU를 사용하고, CPU를 사용 가능하다면 CPU를 사용하도록 설정합니다.

In [ ]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print("cpu와 cuda 중 다음 기기로 학습함:", DEVICE)

# 2. 토치텍스트를 이용한 전처리

여기서는 앞서 배운 토치텍스트를 사용하여 전처리를 진행합니다.

1. 데이터 로드하기 : torchtext.data
torchtext.data의 Field 클래스를 사용하여 영화 리뷰에 대한 객체 TEXT, 레이블을 위한 객체 LABEL을 생성합니다.

In [ ]:
TEXT = data.Field(sequential=True, batch_first=True, lower=True)
LABEL = data.Field(sequential=False, batch_first=True)

2. 데이터 로드 및 분할하기 : torchtext.datasets
torchtext.datasets을 통해 IMDB 리뷰 데이터를 다운로드할 수 있습니다. 데이터를 다운 받는 동시에 훈련 데이터와 테스트 데이터를 분할하고, 각각 trainset, testset에 저장합니다.

In [ ]:
# 전체 데이터를 훈련 데이터와 테스트 데이터를 8:2 비율로 나누기
trainset, testset = datasets.IMDB.splits(TEXT, LABEL)

텍스트와 레이블이 제대로 저장되었는지 확인하기 위해서 trainset.fields를 통해 trainset이 포함하는 각 요소를 확인해봅시다.

In [ ]:
print('trainset의 구성 요소 출력 : ', trainset.fields)

리뷰 데이터가 저장되어져 있는 text 필드와 레이블이 저장되어져 있는 label 필드가 존재합니다. testset.fields도 출력해봅니다.

In [ ]:
print('testset의 구성 요소 출력 : ', testset.fields)

첫번째 훈련 샘플과 해당 샘플에 대한 레이블을 출력해보겠습니다.

In [ ]:
print(vars(trainset[0]))

3. 단어 집합 만들기
이제 단어 집합(vocabulary)을 만들어줍니다. 단어 집합이란 중복을 제거한 총 단어들의 집합을 의미합니다.

In [ ]:
TEXT.build_vocab(trainset, min_freq=5) # 단어 집합 생성
LABEL.build_vocab(trainset)

단어 집합의 크기와 클래스의 개수를 변수에 저장하고 출력해봅니다. 단어 집합의 크기란 결국 중복을 제거한 총 단어의 개수입니다.

In [ ]:
vocab_size = len(TEXT.vocab)
n_classes = 2
print('단어 집합의 크기 : {}'.format(vocab_size))
print('클래스의 개수 : {}'.format(n_classes))

stoi로 단어와 각 단어의 정수 인덱스가 저장되어져 있는 딕셔너리 객체에 접근할 수 있습니다.

In [ ]:
print(TEXT.vocab.stoi)

4. 데이터 로더 만들기
훈련 데이터와 테스트 데이터는 분리하였지만, 이제 검증 데이터를 분리할 차례입니다. 훈련 데이터를 다시 8:2로 분리하여 검증 데이터를 만들겠습니다. 검증 데이터는 valset이라는 변수에 저장합니다.

In [ ]:
trainset, valset = trainset.split(split_ratio=0.8)

토치텍스트는 모든 텍스트를 배치 처리하는 것을 지원하고, 단어를 인덱스 번호로 대체하는 BucketIterator를 제공합니다. BucketIterator는 batch_size, device, shuffle 등의 인자를 받습니다. BATCH_SIZE는 앞서 64로 설정했었습니다.

In [ ]:
train_iter, val_iter, test_iter = data.BucketIterator.splits(
        (trainset, valset, testset), batch_size=BATCH_SIZE,
        shuffle=True, repeat=False)

이제 train_iter, val_iter, test_iter에는 샘플과 레이블이 64개 단위 묶음으로 저장됩니다. 64개씩 묶었을 때 총 배치의 개수가 몇 개가 되는지 출력해봅시다.

In [ ]:
print('훈련 데이터의 미니 배치의 개수 : {}'.format(len(train_iter)))
print('테스트 데이터의 미니 배치의 개수 : {}'.format(len(test_iter)))
print('검증 데이터의 미니 배치의 개수 : {}'.format(len(val_iter)))

첫번째 미니 배치의 크기를 확인해보겠습니다.

In [ ]:
batch = next(iter(train_iter)) # 첫번째 미니배치
print(batch.text.shape)

첫번째 미니 배치의 크기는 64 × 968임을 확인할 수 있습니다. 현재 fix_length를 정해주지 않았으므로 미니 배치 간 샘플들의 길이는 전부 상이합니다. 가령, 두번째 미니 배치의 크기를 확인하면 또 길이가 다름을 확인할 수 있습니다.

In [ ]:
batch = next(iter(train_iter)) # 두번째 미니배치
print(batch.text.shape)

두 개의 미니배치를 꺼내서 크기를 확인하였으므로 이미 꺼낸 두 개의 미니배치를 다시 담기위해 재로드해줍니다.

In [ ]:
train_iter, val_iter, test_iter = data.BucketIterator.splits(
        (trainset, valset, testset), batch_size=BATCH_SIZE,
        shuffle=True, repeat=False)

# 3. RNN 모델 구현하기

모델을 설계합니다.

In [ ]:
class GRU(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2):
        super(GRU, self).__init__()
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim

        self.embed = nn.Embedding(n_vocab, embed_dim)
        self.dropout = nn.Dropout(dropout_p)
        self.gru = nn.GRU(embed_dim, self.hidden_dim,
                          num_layers=self.n_layers,
                          batch_first=True)
        self.out = nn.Linear(self.hidden_dim, n_classes)

    def forward(self, x):
        x = self.embed(x)
        h_0 = self._init_state(batch_size=x.size(0)) # 첫번째 히든 스테이트를 0벡터로 초기화
        x, _ = self.gru(x, h_0)  # GRU의 리턴값은 (배치 크기, 시퀀스 길이, 은닉 상태의 크기)
        h_t = x[:,-1,:] # (배치 크기, 은닉 상태의 크기)의 텐서로 크기가 변경됨. 즉, 마지막 time-step의 은닉 상태만 가져온다.
        self.dropout(h_t)
        logit = self.out(h_t)  # (배치 크기, 은닉 상태의 크기) -> (배치 크기, 출력층의 크기)
        return logit

    def _init_state(self, batch_size=1):
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

In [ ]:
model = GRU(1, 256, vocab_size, 128, n_classes, 0.5).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

모델 훈련 함수를 만듭니다.

In [ ]:
def train(model, optimizer, train_iter):
    model.train()
    for b, batch in enumerate(train_iter):
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1)  # 레이블 값을 0과 1로 변환
        optimizer.zero_grad()

        logit = model(x)
        loss = F.cross_entropy(logit, y)
        loss.backward()
        optimizer.step()

모델 평가 함수를 만듭니다.

In [ ]:
def evaluate(model, val_iter):
    """evaluate model"""
    model.eval()
    corrects, total_loss = 0, 0
    for batch in val_iter:
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1) # 레이블 값을 0과 1로 변환
        logit = model(x)
        loss = F.cross_entropy(logit, y, reduction='sum')
        total_loss += loss.item()
        corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum()
    size = len(val_iter.dataset)
    avg_loss = total_loss / size
    avg_accuracy = 100.0 * corrects / size
    return avg_loss, avg_accuracy

모델을 훈련시킵니다.

In [ ]:
best_val_loss = None
for e in range(1, EPOCHS+1):
    train(model, optimizer, train_iter)
    val_loss, val_accuracy = evaluate(model, val_iter)

    print("[Epoch: %d] val loss : %5.2f | val accuracy : %5.2f" % (e, val_loss, val_accuracy))

    # 검증 오차가 가장 적은 최적의 모델을 저장
    if not best_val_loss or val_loss < best_val_loss:
        if not os.path.isdir("snapshot"):
            os.makedirs("snapshot")
        torch.save(model.state_dict(), './snapshot/txtclassification.pt')
        best_val_loss = val_loss

In [ ]:
model.load_state_dict(torch.load('./snapshot/txtclassification.pt'))
test_loss, test_acc = evaluate(model, test_iter)
print('테스트 오차: %5.2f | 테스트 정확도: %5.2f' % (test_loss, test_acc))

- 참고 : https://github.com/keon/3-min-pytorch
- 토치텍스트 튜토리얼 : https://simonjisu.github.io/nlp/2018/07/18/torchtext.html